# Step 1: Update the Python code to use YugabyteDB
Now that you have the credentials set up, here's how you can update your Python code for connecting to YugabyteDB (PostgreSQL-compatible):

Database Connection Initialization

In [21]:
from dotenv import load_dotenv
import os
import psycopg2

# Load environment variables from .env file
load_dotenv()
# print(f"Host: {os.getenv('POSTGRES_HOST')}")
# print(f"Port: {os.getenv('POSTGRES_PORT')}")
# print(f"Database: {os.getenv('POSTGRES_DB')}")
# print(f"User: {os.getenv('POSTGRES_USER')}")

# Initialize PostgreSQL connection for YugabyteDB
conn = psycopg2.connect(
    dbname=os.getenv('POSTGRES_DB'),
    user=os.getenv('POSTGRES_USER'),
    password=os.getenv('POSTGRES_PASSWORD'),
    host=os.getenv('POSTGRES_HOST'),
    port=os.getenv('POSTGRES_PORT')
)
cursor = conn.cursor()




# step2 : Cell for Creating a User (with duplicate email handling)

In [9]:
import psycopg2
from psycopg2 import errors

# Create a new user with duplicate email handling
def create_user(name, email):
    try:
        cursor.execute(
            "INSERT INTO users (name, email, created_at) VALUES (%s, %s, NOW()) RETURNING id;",
            (name, email)
        )
        conn.commit()
        return cursor.fetchone()[0]
    except errors.UniqueViolation:
        print(f"Error: A user with the email '{email}' already exists.")
        conn.rollback()  # Rollback the transaction if there's a violation
        return None


# step3:performing functions for Reading a User by ID,Updating a User's Email,Deleting a User by ID

In [18]:
# Read user details by ID
def get_user_by_id(user_id):
    cursor.execute("SELECT * FROM users WHERE id = %s;", (user_id,))
    return cursor.fetchone()

# Update user's email
def update_user_email(user_id, new_email):
    try:
        cursor.execute("UPDATE users SET email = %s WHERE id = %s;", (new_email, user_id))
        conn.commit()
        print(f"Email updated successfully for user {user_id}")
    except psycopg2.Error as e:
        conn.rollback()
        print(f"An error occurred: {e}")

# Delete a user by ID
def delete_user(user_id):
    cursor.execute("DELETE FROM users WHERE id = %s;", (user_id,))
    conn.commit()


# Example Usage:
1. Creating a user

In [11]:
user_id = create_user("John Doe", "mohn.doe@example.com")
if user_id:
    print(f"Created User ID: {user_id}")
else:
    print("User creation failed due to duplicate email.")


Created User ID: 201


2.Reading user details:

In [12]:
user = get_user_by_id(user_id)
print(f"User Details: {user}")

User Details: (201, 'John Doe', 'mohn.doe@example.com', datetime.datetime(2024, 9, 13, 13, 10, 34, 46883))


3.Updating a user's email:


In [19]:
try:
    # Ensure there are no pending transactions
    conn.rollback()

    # Update the email
    update_user_email(user_id, "sai.new@example.com")

    # Get and print the updated user details
    updated_user = get_user_by_id(user_id)
    print(f"Updated User Email: {updated_user}")

except psycopg2.Error as e:
    print(f"An error occurred: {e}")

finally:
    # Close the cursor and connection when you're done with all operations
    cursor.close()
    conn.close()

Email updated successfully for user 201
Updated User Email: (201, 'John Doe', 'sai.new@example.com', datetime.datetime(2024, 9, 13, 13, 10, 34, 46883))


4.Deleting a user:

In [ ]:
delete_user(user_id)
print(f"User After Deletion: {get_user_by_id(user_id)}")
